In this notebook, I'm creating dataset for different experiments and also dividing data into train and validation set

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import torch

import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.datasets as dataset
import torchvision

from IPython.core.debugger import set_trace
import matplotlib.pyplot as plt
from PIL import Image

import os
import numpy as np
import pandas as pd

from PIL import ImageFile
# ImageFile.LOAD_TRUNCATED_IMAGES = True
from sklearn.model_selection import train_test_split

from torchvision import models
from torch.optim import lr_scheduler

In [2]:
import cv2
from multiprocessing import pool
from multiprocessing.dummy import Pool as ThreadPool
import os
import shutil

### Looking at the labels

In [72]:
labels = pd.read_csv('data/nih/Data_Entry_2017_correct.csv')

In [73]:
labels.head()

,image_id,pathology,follow_up_#,patient_id,age,gender,view,width,height,pixel_spacing_x,pixel_spacing_y
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143


In [74]:
labels.columns

Index(['image_id', 'pathology', 'follow_up_#', 'patient_id', 'age', 'gender',
       'view', 'width', 'height', 'pixel_spacing_x', 'pixel_spacing_y'],
      dtype='object')

In [75]:
labels.head()

,image_id,pathology,follow_up_#,patient_id,age,gender,view,width,height,pixel_spacing_x,pixel_spacing_y
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143


In [76]:
len(labels)

112120

Reading filenames from downloaded images

In [77]:
fnames = !ls data/nih/images_002_resize/

In [78]:
fnames[:4]

['00001336_000.png',
 '00001337_000.png',
 '00001338_000.png',
 '00001338_001.png']

In [80]:
labels_02 = labels.loc[labels['image_id'].isin(fnames)]

In [113]:
labels_02.shape

(10000, 12)

In [114]:
labels_02.to_csv('tmp.csv',index=False)

In [115]:
from IPython.display import FileLink, FileLinks

In [116]:
FileLink('tmp.csv')

/home/ec2-user/projects/isazi/DL-Xray/tmp.csv

In [82]:
labels_02.patient_id.nunique()

2588

In [83]:
pathology = [
    'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass',
    'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation', 'Edema',
    'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia'
]

In [84]:
pathology_dict = {}

In [85]:
for i in pathology:
    pathology_dict[i] = np.sum(
        labels_02['pathology'].apply(lambda x: i in x))

In [86]:
pathology_dict

{'Atelectasis': 890,
 'Cardiomegaly': 200,
 'Consolidation': 352,
 'Edema': 106,
 'Effusion': 895,
 'Emphysema': 193,
 'Fibrosis': 207,
 'Hernia': 19,
 'Infiltration': 1430,
 'Mass': 309,
 'Nodule': 452,
 'Pleural_Thickening': 303,
 'Pneumonia': 119,
 'Pneumothorax': 471}

In [87]:
labels_02.groupby('view').size()

view
AP    3408
PA    6592
dtype: int64

In [88]:
labels_02.head()

,image_id,pathology,follow_up_#,patient_id,age,gender,view,width,height,pixel_spacing_x,pixel_spacing_y
4999,00001336_000.png,No Finding,0,1336,16,M,PA,2048,2500,0.171,0.171
5000,00001337_000.png,No Finding,0,1337,53,F,PA,2992,2991,0.143,0.143
5001,00001338_000.png,No Finding,0,1338,60,M,PA,2992,2991,0.143,0.143
5002,00001338_001.png,No Finding,1,1338,61,M,AP,3056,2544,0.139,0.139
5003,00001338_002.png,Infiltration,2,1338,61,M,AP,3056,2544,0.139,0.139


### Creating dataset for Atelectasis experiment

In [89]:
labels_02 = labels_02.assign(
    Pneumothorax_indicator=labels_02.pathology.apply(
        lambda x: 1 if 'Pneumothorax' in x else 0))

In [112]:
labels_02

,image_id,pathology,follow_up_#,patient_id,age,gender,view,width,height,pixel_spacing_x,pixel_spacing_y,Pneumothorax_indicator
4999,00001336_000.png,No Finding,0,1336,16,M,PA,2048,2500,0.171000,0.171000,0
5000,00001337_000.png,No Finding,0,1337,53,F,PA,2992,2991,0.143000,0.143000,0
5001,00001338_000.png,No Finding,0,1338,60,M,PA,2992,2991,0.143000,0.143000,0
5002,00001338_001.png,No Finding,1,1338,61,M,AP,3056,2544,0.139000,0.139000,0
5003,00001338_002.png,Infiltration,2,1338,61,M,AP,3056,2544,0.139000,0.139000,0
5004,00001338_003.png,Infiltration,3,1338,61,M,PA,2021,2021,0.194311,0.194311,0
5005,00001339_000.png,No Finding,0,1339,51,M,PA,2992,2991,0.143000,0.143000,0
5006,00001340_000.png,No Finding,0,1340,32,F,PA,2992,2991,0.143000,0.143000,0
5007,00001341_000.png,No Finding,0,1341,54,F,PA,2992,2569,0.143000,0.143000,0
5008,00001342_000.png,Infiltration,0,1342,63,F,AP,2500,2048,0.171000,0.171000,0


In [91]:
pos_PA = labels_02.loc[(labels_02.Pneumothorax_indicator == 1)
                          & (labels_02.view == 'PA')]['patient_id']

pos_AP = labels_02.loc[(labels_02.Pneumothorax_indicator == 1)
                          & (labels_02.view == 'AP')]['patient_id']

neg_PA = labels_02.loc[(labels_02.Pneumothorax_indicator == 0)
                          & (labels_02.view == 'PA')]['patient_id']

neg_AP = labels_02.loc[(labels_02.Pneumothorax_indicator == 0)
                          & (labels_02.view == 'AP')]['patient_id']

In [92]:
val_pid = []
np.random.seed(seed=4)
val_pid.extend(np.random.choice(pos_PA, pos_PA.shape[0] // 25))
val_pid.extend(np.random.choice(pos_AP, pos_AP.shape[0] // 25))
val_pid.extend(np.random.choice(neg_PA, neg_PA.shape[0] // 80))
val_pid.extend(np.random.choice(neg_AP, neg_AP.shape[0] // 80))
val_pid = list(set(val_pid))

In [93]:
val_img_df = labels_02.loc[labels_02.patient_id.isin(val_pid)]

In [94]:
trn_img_df = labels_02.loc[~labels_02.patient_id.isin(val_pid)]

In [95]:
val_img_df.shape

(1460, 12)

In [96]:
trn_img_df.shape

(8540, 12)

In [98]:
val_img_df.Pneumothorax_indicator.sum()

112

In [99]:
trn_img_df.Pneumothorax_indicator.sum()

359

In [55]:
src = 'data/nih/images_002_resize/'

In [109]:
files = list(trn_img_df.loc[trn_img_df.Pneumothorax_indicator==0]['image_id'])

In [110]:
dest = 'data/nih/Pneumothorax/train/neg/'

In [111]:
for file_name in files:
    full_file_name = os.path.join(src, file_name)
    if (os.path.isfile(full_file_name)):
        shutil.copy(full_file_name, dest)

### Single channel to 3 channel

In [3]:
src = 'data/nih/images_resized_01-04/'

In [7]:
fnames = os.listdir(src)
fpath = [os.path.join(src,f) for f in fnames]

In [20]:
def single_to_three_channel(src_path,dest_path):
    fnames = os.listdir(src_path)
    fpath = [os.path.join(src_path,f) for f in fnames]
    os.makedirs(dest_path,exist_ok=True)
    for i,n in enumerate(fpath):
        img = plt.imread(n)
        img_new = np.stack((img,img,img),-1)
        plt.imsave(os.path.join(dest_path,fnames[i]),img_new)